Gerekli kütüphanelerin yüklenmesi

In [1]:
# kütüphaneleri import edelim
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter

# pythondaki uyarıları kapatmak için
import warnings
warnings.filterwarnings('ignore')
import missingno as msno
from datetime import date
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
pd.set_option('display.width', 500)

Verinin yüklenmesi

In [ ]:
df= pd.read_excel('MarketSales.xlsx')

Veriye ilk bakış 

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

In [ ]:
df.isnull().sum().sort_values(ascending=False)

Kolon isimlerin türkçeleştirilmesi Power BI de rapor hazırlancaksa

In [ ]:
df = df.rename(columns={
    'ID': 'ID',
    'ITEMCODE': 'ÜrünKodu',
    'ITEMNAME': 'ÜrünAdı',
    'FICHENO': 'FişNo',
    'DATE_': 'Tarih',
    'AMOUNT': 'Miktar',
    'PRICE': 'Fiyat',
    'LINENETTOTAL': 'ToplamNetSatış',
    'LINENET': 'NetSatış',
    'BRANCHNR': 'ŞubeNo',
    'BRANCH': 'Şube',
    'SALESMAN': 'SatışElemanı',
    'CITY': 'Şehir',
    'REGION': 'Bölge',
    'LATITUDE': 'Enlem',
    'LONGITUDE': 'Boylam',
    'CLIENTCODE': 'MüşteriKodu',
    'CLIENTNAME': 'MüşteriAdı',
    'BRANDCODE': 'MarkaKodu',
    'BRAND': 'Marka',
    'CATEGORY_NAME1': 'KategoriAdı1',
    'CATEGORY_NAME2': 'KategoriAdı2',
    'CATEGORY_NAME3': 'KategoriAdı3',
    'STARTDATE': 'BaşlangıçTarihi',
    'ENDDATE': 'BitişTarihi',
    'GENDER': 'Cinsiyet'
})

Eksik Verilerin Analiz Edilmesi

MarkaKodu, Marka, KategoriAdı1,KategoriAdı2,KategoriAdı3
Bu beş kolon birbiri ile baglantılı oldugu için beş kolonunda boş oldugu sütünları veriden düşürüyoruz çünkü tahmin edilecek deger bulunamadıgı için

In [ ]:
total_null_in_selected_columns = df[['MarkaKodu', 'Marka', 'KategoriAdı1', 'KategoriAdı2','KategoriAdı3']].isnull().all(axis=1).sum()
total_null_in_selected_columns

In [ ]:
df.dropna(subset=['MarkaKodu', 'Marka', 'KategoriAdı1', 'KategoriAdı2', 'KategoriAdı3'], how='all', inplace=True)


In [ ]:
df.isnull().sum().sort_values(ascending=False)

Aynı şekilde bu üç kolondaki degerlerde birbirleriyle baglantılı oldugu için hepsinin boş oldugu satırları düşürüyoruz

In [ ]:
total_null_in_selected_columns2 = df[[ 'MüşteriAdı', 'MüşteriKodu','Cinsiyet']].isnull().all(axis=1).sum()
total_null_in_selected_columns2

In [ ]:
df.dropna(subset=[ 'MüşteriAdı', 'MüşteriKodu','Cinsiyet'], how='all', inplace=True)

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
total_null_in_selected_columns3 = df[[ 'MüşteriAdı', 'Cinsiyet']].isnull().all(axis=1).sum()
total_null_in_selected_columns3

In [ ]:
df.dropna(subset=[ 'MüşteriAdı','Cinsiyet'], how='all', inplace=True)

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
total_null_in_selected_columns4 = df[[ 'Marka', 'MarkaKodu']].isnull().all(axis=1).sum()
total_null_in_selected_columns4

Marka kolunu doldurmak için marka kolonuna baglı olan diger kolonlarla yeni bir tablo oluşturalım veri çercevesini dahi iyi görebilmek için

In [ ]:
# Belirli sütunlarda yalnızca NaN değerlere sahip olan satırları seçme
nan_satirlar = df[df['Marka'].isna() & df['MarkaKodu'].isna()]

# İstenilen sütunları seçme
secilen_sutunlar = ['Marka', 'MarkaKodu', 'KategoriAdı1', 'KategoriAdı2', 'KategoriAdı3']
secilen_veri = df.loc[nan_satirlar.index, secilen_sutunlar]

# Yeni DataFrame oluşturma
yeni_df = secilen_veri.copy()

# İşlenmiş DataFrame'i kontrol etme
print(yeni_df)

# Excel dosyası adı ve yolu
dosya_yolu = "yeni_veri.xlsx"

# DataFrame'i Excel dosyasına kaydetme
yeni_df.to_excel(dosya_yolu, index=False)

print("Excel dosyası başarıyla kaydedildi.")



In [ ]:
yeni_df.isnull().sum().sort_values(ascending=False)

In [ ]:
yeni_df.dropna(subset=['KategoriAdı3'], how='all', inplace=True)


In [ ]:
print(yeni_df)

In [ ]:
a=yeni_df.groupby([ 'KategoriAdı1','KategoriAdı2','KategoriAdı3']).size()
tablo=a.reset_index(name='sayı')
print(tablo)

In [ ]:
# 'sayı' sütununun toplamını al
toplam_sayi = tablo['sayı'].sum()
print("Toplam Sayı:", toplam_sayi)


Gruplandırark oluşturdugumuz tablodan marka degerleri kategorik degişken oldugu için mode ile dolduralım
   gruplandırma yapılmasının amacı kategeorilerin bir biriyle baglı olması ve onların marka baglı olması

0.SATIR İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'BEBEK') & \
         (df['KategoriAdı2'] == 'BEBEK BAKIM') & \
         (df['KategoriAdı3'] == 'BEBEK BAKIM MALZEMELERİ')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

1.SATIR İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'BEBEK') & \
         (df['KategoriAdı2'] == 'BEBEK BAKIM') & \
         (df['KategoriAdı3'] == 'PUDRALAR')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

2.SATIR İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'DETERJAN TEMİZLİK') & \
         (df['KategoriAdı2'] == 'EV TEMİZLEYİCİ') & \
         (df['KategoriAdı3'] == 'MUTFAK BANYO TEMİZLEYİCİ')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

3.SATIR İÇİN 

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre=                (df['KategoriAdı1'] == 'ET TAVUK') & \
                       (df['KategoriAdı2'] == 'ET ŞARKÜTERİ') & \
                       (df['KategoriAdı3'] == 'KAVURMA')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

4.SATIR İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'ET TAVUK') & \
         (df['KategoriAdı2'] == 'ET ŞARKÜTERİ') & \
         (df['KategoriAdı3'] == 'PASTIRMA')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)


5.SATIR İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'ET TAVUK') & \
         (df['KategoriAdı2'] == 'ET ŞARKÜTERİ') & \
         (df['KategoriAdı3'] == 'SALAM')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

6.SATIR İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'ET TAVUK') & \
         (df['KategoriAdı2'] == 'ET ŞARKÜTERİ') & \
         (df['KategoriAdı3'] == 'SOSİS')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

7.SATIR İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'ET TAVUK') & \
         (df['KategoriAdı2'] == 'KIRMIZI ET') & \
         (df['KategoriAdı3'] == 'DANA')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

8.SATIR İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'EV') & \
         (df['KategoriAdı2'] == 'MUTFAK EŞYA GEREÇLERİ') & \
         (df['KategoriAdı3'] == 'CAM EL ALETLERİ')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

9.SATIR İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'EV') & \
         (df['KategoriAdı2'] == 'MUTFAK EŞYA GEREÇLERİ') & \
         (df['KategoriAdı3'] == 'PLASTİK EV GEREÇLERİ')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

10 SATIR İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'EV') & \
         (df['KategoriAdı2'] == 'MUTFAK EŞYA GEREÇLERİ') & \
         (df['KategoriAdı3'] == 'PİŞİRME GEREÇLERİ')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

11.SATIR İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'GIDA') & \
         (df['KategoriAdı2'] == 'BÜSKİVİ ÇEREZ') & \
         (df['KategoriAdı3'] == 'KURUYEMİŞ')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

12.SATIR İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'GIDA') & \
         (df['KategoriAdı2'] == 'HAZIR YEMEKLER') & \
         (df['KategoriAdı3'] == 'MEZE')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

13.SATIR İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'GIDA') & \
         (df['KategoriAdı2'] == 'HAZIR YEMEKLER') & \
         (df['KategoriAdı3'] == 'SİRKE SOS')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

14.SATIR İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'GIDA') & \
         (df['KategoriAdı2'] == 'UNLU MAMÜLLER') & \
         (df['KategoriAdı3'] == 'EKMEK')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

15.SATIR İÇİN

In [ ]:
filtre = (df['KategoriAdı1'] == 'GIDA') & \
         (df['KategoriAdı2'] == 'UNLU MAMÜLLER') & \
         (df['KategoriAdı3'] == 'KURABİYE GALETE GRİSİNİ')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

16.SATIR İÇİN

In [ ]:
 # Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'GIDA') & \
                       (df['KategoriAdı2'] == 'UNLU MAMÜLLER') & \
                       (df['KategoriAdı3'] == 'TATLILAR')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

17.SATIR İÇİN

In [ ]:
 # Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'GIDA') & \
                       (df['KategoriAdı2'] == 'UNLU MAMÜLLER') & \
                       (df['KategoriAdı3'] == 'TOZ TATLILAR')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

18.SATIR İÇİN

In [ ]:
 # Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'GIDA')  & \
                       (df['KategoriAdı2'] == 'UNLU MAMÜLLER')  & \
                       (df['KategoriAdı3'] == 'UN')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

19.SATIR İÇİN

In [ ]:
 # Belirli şartları sağlayan satırları filtreleme
filtre=(df['KategoriAdı1'] == 'KOZMETİK') & \
                       (df['KategoriAdı2'] == 'MAKYAJ MALZEMELERİ') & \
                       (df['KategoriAdı3'] == 'MANİKUR PEDİKÜR ÜRÜNLERİ')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

20.SATIR İÇİN

In [ ]:
 # Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'KOZMETİK') & \
                       (df['KategoriAdı2'] == 'PARFÜM DEODORANT') & \
                       (df['KategoriAdı3'] == 'KOLANYALAR')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

21.SATIR İÇİN

In [ ]:
 # Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'SÜT KAHVALTILIK') & \
                       (df['KategoriAdı2'] == 'KAHVALTILIK') & \
                       (df['KategoriAdı3'] == 'HELVA')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

22.SATIR İÇİN

In [ ]:
 # Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'SÜT KAHVALTILIK') & \
                       (df['KategoriAdı2'] == 'KAHVALTILIK') & \
                       (df['KategoriAdı3'] == 'KREM ÇİKOLATA')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

23.SATIR İÇİN

In [ ]:
 # Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'SÜT KAHVALTILIK') & \
                       (df['KategoriAdı2'] == 'KAHVALTILIK') & \
                       (df['KategoriAdı3'] == 'TAHİN PEKMEZ')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

24.SATIR İÇİN

In [ ]:
 # Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'SÜT KAHVALTILIK') & \
                       (df['KategoriAdı2'] == 'PEYNİR') & \
                       (df['KategoriAdı3'] == 'BEYAZ PEYNİR')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

25.SATIR İÇİN

In [ ]:
 # Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'SÜT KAHVALTILIK') & \
                       (df['KategoriAdı2'] == 'PEYNİR') & \
                       (df['KategoriAdı3'] == 'ÖZEL-KREM PEYNİR')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

26.SATIR İÇİN

In [ ]:
 # Belirli şartları sağlayan satırları filtreleme
filtre=(df['KategoriAdı1'] == 'SÜT KAHVALTILIK') & \
                       (df['KategoriAdı2'] == 'SÜTLÜ TATLILAR DONDURMA') & \
                       (df['KategoriAdı3'] == 'DONDURMALAR')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

27.SATIR İÇİN

In [ ]:
 # Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'SÜT KAHVALTILIK') & \
                       (df['KategoriAdı2'] == 'ZEYTİN') & \
                       (df['KategoriAdı3'] == 'SİYAH ZEYTİN')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

28.SATIR İÇİN

In [ ]:
 # Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'İÇECEK') & \
                       (df['KategoriAdı2'] == 'GAZSIZ İÇECEK') & \
                       (df['KategoriAdı3'] == 'SPORCU İÇECEĞİ')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

29.SATIR İÇİN

In [ ]:
 # Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'İÇECEK') & \
                       (df['KategoriAdı2'] == 'GAZSIZ İÇECEK') & \
                       (df['KategoriAdı3'] == 'TOZ İÇECEK')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

30.SATIR İÇİN

In [ ]:
 # Belirli şartları sağlayan satırları filtreleme
filtre= (df['KategoriAdı1'] == 'İÇECEK') & \
                       (df['KategoriAdı2'] == 'SU MADENSUYU') & \
                       (df['KategoriAdı3'] == 'SU')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)

31.SATIR İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'İÇECEK') & \
         (df['KategoriAdı2'] == 'ÇAY KAHVE') & \
         (df['KategoriAdı3'] == 'SEKER TATLANDIRICI')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol1)

# En sık geçen markayı bulma
en_sik_marka = df.loc[filtre, 'Marka'].mode()[0]
print(en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'Marka'] = df.loc[filtre, 'Marka'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'Marka'].isnull().sum()
print(kontrol2)


In [ ]:
df.isnull().sum()

In [ ]:
df.dropna(subset=['Marka'], inplace=True)

In [ ]:
df.isnull().sum()

In [ ]:
df.groupby('Marka')['MarkaKodu'].first()

Makraya göre markakodunun doldurulması

In [ ]:
# Her bir Marka için benzersiz Marka Kodlarını al
marka_kodlari = df.groupby('Marka')['MarkaKodu'].first()

# Marka Kodlarını eksik değerleri doldurmak için kullan
df['MarkaKodu'] = df['MarkaKodu'].fillna(df['Marka'].map(marka_kodlari))


In [ ]:
df.isnull().sum()

In [ ]:
total_null_in_selected_columns3 = df[['KategoriAdı2','KategoriAdı3']].isnull().all(axis=1).sum()
total_null_in_selected_columns3

In [ ]:
df.dropna(subset=['KategoriAdı2'], inplace=True)

In [ ]:
df.isnull().sum()

KategoriAdı1,KategoriAdı2,KategoriAdı3 kolonların doldurulması

In [199]:
# Gruplama için sütunları birleştirme
liste = [df['KategoriAdı1'], df['KategoriAdı2'], df['KategoriAdı3']]
gruplanmis_veri = df.groupby(liste)

# Sayıları hesaplama
sayilar = gruplanmis_veri.size()

# İndeksleri sütunlara dönüştürme ve tablo oluşturma
tablo = sayilar.reset_index(name='Sayı')

# Sonucu görüntüleme
print(tablo)



          KategoriAdı1             KategoriAdı2              KategoriAdı3   Sayı
0                BEBEK              BEBEK BAKIM      BEBEK BAKIM ARAÇLARI     38
1                BEBEK              BEBEK BAKIM   BEBEK BAKIM MALZEMELERİ    487
2                BEBEK              BEBEK BAKIM          LOSYON VE SÜTLER      2
3                BEBEK              BEBEK BAKIM                  PUDRALAR      7
4                BEBEK              BEBEK BAKIM                 SABUNLARI     26
5                BEBEK              BEBEK BAKIM                ŞAMPUANLAR     87
6                BEBEK         HAZIR YEMEK MAMA         BEBEK BİSKÜVİLERİ   2075
7                BEBEK         HAZIR YEMEK MAMA          BİBERON MAMALARI    160
8                BEBEK         HAZIR YEMEK MAMA             HAZIR MAMALAR    672
9                BEBEK            ÇOCUK BEZLERİ                    BEZLER   1779
10   DETERJAN TEMİZLİK           BULAŞIK YIKAMA               ELDE YIKAMA   3156
11   DETERJAN TEMİZLİK      

0,1,2,3,4 VE 5 İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'BEBEK') & \
         (df['KategoriAdı2'] == 'BEBEK BAKIM')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)


6,7 VE 8 İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'BEBEK') & \
         (df['KategoriAdı2'] == 'HAZIR YEMEK MAMA')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)


10 VE 11 İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'DETERJAN TEMİZLİK') & \
         (df['KategoriAdı2'] == 'BULAŞIK YIKAMA')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)


12:21 İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'DETERJAN TEMİZLİK') & \
         (df['KategoriAdı2'] == 'EV TEMİZLEYİCİ')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)


22,23 VE 24 İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'DETERJAN TEMİZLİK') & \
         (df['KategoriAdı2'] == 'EV TEMİZLİK GEREÇLERİ')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)


25:31 İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'DETERJAN TEMİZLİK') & \
         (df['KategoriAdı2'] == 'ÇAMAŞIR YIKAMA')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)


32:36 İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'ET TAVUK') & \
         (df['KategoriAdı2'] == 'ET ŞARKÜTERİ')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)


37 VE 38 İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'ET TAVUK') & \
         (df['KategoriAdı2'] == 'KIRMIZI ET')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)


39 İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'ET TAVUK') & \
         (df['KategoriAdı2'] == 'KÜMES HAYVANLARI')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)


40 VE 41 İÇİN 

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'EV') & \
         (df['KategoriAdı2'] == 'ELEKTRONİK')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)


42 İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'EV') & \
         (df['KategoriAdı2'] == 'EV TEKSTİL')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)


43:49 İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'EV') & \
         (df['KategoriAdı2'] == 'MUTFAK EŞYA GEREÇLERİ')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)


In [ ]:
50:52

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'EV') & \
         (df['KategoriAdı2'] == 'İÇ GİYİM AKSESUAR')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)


53 VE 54 İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'GIDA') & \
         (df['KategoriAdı2'] == 'BAKLİYAT')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

55 İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'GIDA') & \
         (df['KategoriAdı2'] == 'BAYRAMLIK')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

56:60 İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'GIDA') & \
         (df['KategoriAdı2'] == 'BÜSKİVİ ÇEREZ')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

61 VE 62 İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'GIDA') & \
         (df['KategoriAdı2'] == 'CORBA BULYON')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

63:70 İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'GIDA') & \
         (df['KategoriAdı2'] == 'HAZIR YEMEKLER')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

71 İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'GIDA') & \
         (df['KategoriAdı2'] == 'MAKARNA')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

72 VE 73 İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'GIDA') & \
         (df['KategoriAdı2'] == 'SAKIZ SEKERLEME')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

74 VE 79 İÇİN 

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'GIDA') & \
         (df['KategoriAdı2'] == 'SIVI YAĞLAR')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

80:83 İÇİN 

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'GIDA') & \
         (df['KategoriAdı2'] == 'TUZ BAHARAT')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

84:89 İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'GIDA') & \
         (df['KategoriAdı2'] == 'UNLU MAMÜLLER')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

90:93 İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'KOZMETİK') & \
         (df['KategoriAdı2'] == 'AĞIZ BAKIM')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

94:96 İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'KOZMETİK') & \
         (df['KategoriAdı2'] == 'DUŞ BANYO')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

97:101 İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'KOZMETİK') & \
         (df['KategoriAdı2'] == 'MAKYAJ MALZEMELERİ')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

102:106 İÇİN

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'KOZMETİK') & \
         (df['KategoriAdı2'] == 'PARFÜM DEODORANT')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

107:110 İÇİN 

In [ ]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'KOZMETİK') & \
         (df['KategoriAdı2'] == 'SAÇ BAKIM')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

111:114 İÇİN

In [200]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'KOZMETİK') & \
         (df['KategoriAdı2'] == 'TRAŞ MALZEMELERİ')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

Eksik değerlerin sayısı: 0
En sık geçen marka: TRAŞ ALETLERİ
Eksik değerlerin sayısı (doldurulduktan sonra): 0


115 VE 116 İÇİN

In [201]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'KOZMETİK') & \
         (df['KategoriAdı2'] == 'VUCUT EL BAKIMI')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

Eksik değerlerin sayısı: 0
En sık geçen marka: KREMLER
Eksik değerlerin sayısı (doldurulduktan sonra): 0


117 İÇİN

In [202]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'KOZMETİK') & \
         (df['KategoriAdı2'] == 'YÜZ BAKIMI')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

Eksik değerlerin sayısı: 0
En sık geçen marka: YÜZ TEMİZLEME ÜRÜNLERİ
Eksik değerlerin sayısı (doldurulduktan sonra): 0


118 :123 İÇİN

In [204]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'SÜT KAHVALTILIK') & \
         (df['KategoriAdı2'] == 'KAHVALTILIK')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

Eksik değerlerin sayısı: 384
En sık geçen marka: KREM ÇİKOLATA
Eksik değerlerin sayısı (doldurulduktan sonra): 0


124:126 İÇİN 

In [205]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'SÜT KAHVALTILIK') & \
         (df['KategoriAdı2'] == 'PEYNİR')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

Eksik değerlerin sayısı: 1657
En sık geçen marka: BEYAZ PEYNİR
Eksik değerlerin sayısı (doldurulduktan sonra): 0


127:129 İÇİN

In [206]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'SÜT KAHVALTILIK') & \
         (df['KategoriAdı2'] == 'SÜT')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

Eksik değerlerin sayısı: 0
En sık geçen marka: PASTÖRİZE SÜT
Eksik değerlerin sayısı (doldurulduktan sonra): 0


130:132 İÇİN 

In [207]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'SÜT KAHVALTILIK') & \
         (df['KategoriAdı2'] == 'SÜTLÜ TATLILAR DONDURMA')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

Eksik değerlerin sayısı: 394
En sık geçen marka: SÜTLÜ TATLILAR
Eksik değerlerin sayısı (doldurulduktan sonra): 0


133:134 İÇİN

In [208]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'SÜT KAHVALTILIK') & \
         (df['KategoriAdı2'] == 'TEREYAĞ MARGARİN')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

Eksik değerlerin sayısı: 0
En sık geçen marka: MARGARİNLER
Eksik değerlerin sayısı (doldurulduktan sonra): 0


135 VE 136 İÇİN

In [210]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'SÜT KAHVALTILIK') & \
         (df['KategoriAdı2'] == 'YOĞURT')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

Eksik değerlerin sayısı: 0
En sık geçen marka: YOĞURTLAR
Eksik değerlerin sayısı (doldurulduktan sonra): 0


137 İÇİN

In [212]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'SÜT KAHVALTILIK') & \
         (df['KategoriAdı2'] == 'YUMURTA')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

Eksik değerlerin sayısı: 9870
En sık geçen marka: NORMAL YUMURTA
Eksik değerlerin sayısı (doldurulduktan sonra): 0


138 VE 139 İÇİN

In [211]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'SÜT KAHVALTILIK') & \
         (df['KategoriAdı2'] == 'ZEYTİN')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

Eksik değerlerin sayısı: 2841
En sık geçen marka: SİYAH ZEYTİN
Eksik değerlerin sayısı (doldurulduktan sonra): 0


140:142 İÇİN

In [213]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'İÇECEK') & \
         (df['KategoriAdı2'] == 'GAZLI İÇECEK')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

Eksik değerlerin sayısı: 115
En sık geçen marka: COLA
Eksik değerlerin sayısı (doldurulduktan sonra): 0


142:148 İÇİN

In [214]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'İÇECEK') & \
         (df['KategoriAdı2'] == 'GAZSIZ İÇECEK')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

Eksik değerlerin sayısı: 0
En sık geçen marka: MEYVE SUYU
Eksik değerlerin sayısı (doldurulduktan sonra): 0


149 İÇİN

In [216]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'İÇECEK') & \
         (df['KategoriAdı2'] == 'GÜNLÜK İÇECEK')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

Eksik değerlerin sayısı: 0
En sık geçen marka: AYRAN 
Eksik değerlerin sayısı (doldurulduktan sonra): 0


150 VE 151 İÇİN

In [217]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'İÇECEK') & \
         (df['KategoriAdı2'] == 'SU MADENSUYU')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

Eksik değerlerin sayısı: 0
En sık geçen marka: SU
Eksik değerlerin sayısı (doldurulduktan sonra): 0


In [218]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'İÇECEK') & \
         (df['KategoriAdı2'] == 'ÇAY KAHVE')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0]
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

Eksik değerlerin sayısı: 0
En sık geçen marka: KAHVE
Eksik değerlerin sayısı (doldurulduktan sonra): 0


In [219]:
df.isnull().sum()

ID                      0
ÜrünKodu                0
ÜrünAdı                 0
FişNo                   0
Tarih                   0
Miktar                  0
Fiyat                   0
ToplamNetSatış          0
NetSatış                0
ŞubeNo                  0
Şube                    0
SatışElemanı            0
Şehir                   0
Bölge                   0
Enlem                   0
Boylam                  0
MüşteriKodu             0
MüşteriAdı              0
MarkaKodu               0
Marka                   0
KategoriAdı1            0
KategoriAdı2            0
KategoriAdı3       154024
BaşlangıçTarihi         0
BitişTarihi             0
Cinsiyet                0
dtype: int64

In [ ]:
df.drop('KategoriAdı3', axis=1)

KategoriAdı3 kolonunu düşürmek zorunda kalıyoruz çünkü bu projenin amacı Power BI de rapor hazırlamak oldugu için KategoriAdı3 deki nan degerler silindigi zaman int degerler de düşürelecegi için toplam tutar ,toplam miktar rakamlarda büyük oranda farklılık olurdu yukararıdeki kodlarda doldurulacak max kategaori3 kodları verilmiştir rapor hazırlama degil ML için kullanılacagı zaman kodllar kullanılabilir 

In [248]:
#gruplanmis_df = df.groupby(['ÜrünAdı','KategoriAdı1', 'KategoriAdı2','' df['KategoriAdı3'].isnull()]).size().reset_index(name='Sayı')
#print(gruplanmis_df)

# Create a new column indicating missing values in 'KategoriAdı3'
df['KategoriAdı3_isnull'] = df['KategoriAdı3'].isnull()

# Group by and count occurrences
gruplanmis_df = df.groupby(['ÜrünAdı', 'KategoriAdı1', 'KategoriAdı2', 'KategoriAdı3_isnull']).size().reset_index(name='Sayı')
# Call the reset_index method to create a new DataFrame
tablo2 = gruplanmis_df.reset_index()

# Print the grouped DataFrame
print(tablo2)


      index                                            ÜrünAdı       KategoriAdı1             KategoriAdı2  KategoriAdı3_isnull  Sayı
0         0                           8*4 150 ML DEO DİSCOVERY           KOZMETİK         PARFÜM DEODORANT                False     1
1         1                8*4 150 ML DEO FOR MEN URBAN SPIRIT           KOZMETİK         PARFÜM DEODORANT                False     3
2         2                8*4 150 ML DEO FORMEN PLAY THE GAMR           KOZMETİK         PARFÜM DEODORANT                False     2
3         3                          8*4 150 ML DEO HOLLY WOOD           KOZMETİK         PARFÜM DEODORANT                False     5
4         4                          8*4 150 ML DEO LEMON RUSH           KOZMETİK         PARFÜM DEODORANT                False     3
5         5                       8*4 150 ML DEO MODERN CHARME           KOZMETİK         PARFÜM DEODORANT                False     3
6         6                  8*4 150 ML DEO PRO ACTIVE MEXIMUM

In [233]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'GIDA') & \
         (df['KategoriAdı2'] == 'ÇİKOLATA GOFRET')

# Eksik değerlerin sayısını kontrol et
kontrol1 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı:", kontrol1)

# En sık geçen Kategori Adı3  bulma
en_sik_marka = df.loc[filtre, 'KategoriAdı3'].mode()[0] if not df.loc[filtre, 'KategoriAdı3'].mode().empty else None
print("En sık geçen marka:", en_sik_marka)

# Eksik değerleri doldurma
df.loc[filtre, 'KategoriAdı3'] = df.loc[filtre, 'KategoriAdı3'].fillna(en_sik_marka)

# Kontrol etme
kontrol2 = df.loc[filtre, 'KategoriAdı3'].isnull().sum()
print("Eksik değerlerin sayısı (doldurulduktan sonra):", kontrol2)

SyntaxError: invalid non-printable character U+00A0 (859886775.py, line 11)

In [231]:
df.isnull().sum()

ID                      0
ÜrünKodu                0
ÜrünAdı                 0
FişNo                   0
Tarih                   0
Miktar                  0
Fiyat                   0
ToplamNetSatış          0
NetSatış                0
ŞubeNo                  0
Şube                    0
SatışElemanı            0
Şehir                   0
Bölge                   0
Enlem                   0
Boylam                  0
MüşteriKodu             0
MüşteriAdı              0
MarkaKodu               0
Marka                   0
KategoriAdı1            0
KategoriAdı2            0
KategoriAdı3       154024
BaşlangıçTarihi         0
BitişTarihi             0
Cinsiyet                0
dtype: int64

In [ ]:
import locale

# Türkçe yerel ayarları yükleme
locale.setlocale(locale.LC_TIME, 'tr_TR.UTF-8')


# Yıl, ay, gün ve haftanın günü sütunlarını oluşturma
df['Yil'] = pd.to_datetime(df['BaşlangıçTarihi']).dt.year
df['Ay'] = pd.to_datetime(df['BaşlangıçTarihi']).dt.month
df['Gun'] = pd.to_datetime(df['BaşlangıçTarihi']).dt.day

# Haftanın gününü Türkçe olarak almak için özel bir fonksiyon tanımlama
def haftanin_gunu_tr(tarih):
    gunler = ['Pazartesi', 'Salı', 'Çarşamba', 'Perşembe', 'Cuma', 'Cumartesi', 'Pazar']
    return gunler[tarih.weekday()]

df['HaftaninGunu'] = pd.to_datetime(df['BaşlangıçTarihi']).apply(haftanin_gunu_tr)

# Sonuçları görüntüleme
df.head()

In [256]:
# Belirli şartları sağlayan satırları filtreleme
filtre = (df['KategoriAdı1'] == 'GIDA') & (df['KategoriAdı2'] == 'ÇİKOLATA GOFRET')

# Filtreye göre gruplama ve boyutunu alma
gruplanmis_df = df[filtre].groupby(['KategoriAdı1', 'KategoriAdı2','KategoriAdı3']).size().reset_index(name='Sayı')
print(gruplanmis_df)

Empty DataFrame
Columns: [KategoriAdı1, KategoriAdı2, KategoriAdı3, Sayı]
Index: []


In [ ]:
# Ay sütununu metinsel formata dönüştürme
aylar = {
    1: 'Ocak',
    2: 'Şubat',
    3: 'Mart',
    4: 'Nisan',
    5: 'Mayıs',
    6:'Haziran',
    7:'Temmuz',
    8:'Agustos',
    9:'Eylül',
    10:'Ekim',
    11:'Kasım',
    12:'Aralık'
    # Diğer ayları da ekleyebilirsiniz
}
df['Ay'] = df['Ay'].map(aylar)

# Sonuçları görüntüleme
df.head()

In [13]:
df2= pd.read_csv('son_veri.csv')

In [14]:
df2.head()

,ID,ÜrünKodu,ÜrünAdı,FişNo,Tarih,Miktar,Fiyat,ToplamNetSatış,NetSatış,ŞubeNo,Şube,SatışElemanı,Şehir,Bölge,Enlem,Boylam,MüşteriKodu,MüşteriAdı,MarkaKodu,Marka,KategoriAdı1,KategoriAdı2,KategoriAdı3,BaşlangıçTarihi,BitişTarihi,Cinsiyet,Yil,Ay,Gun,HaftaninGunu
0,11738.000,5863.000,SPRITE 1 LT LIMON AROMALI GAZOZ,18456,2017-01-07,1.000,2.000,2.000,1.850,52.000,Kocaeli Subesi,Eyüp CİNE,Kocaeli,Marmara,40.853,29.881,467369,Sercan KIZILOK,156,SPRİTE,İÇECEK,GAZLI İÇECEK,GAZOZ,2017-01-08 16:16:11,2017-01-08 16:17:13,E,2017,Ocak,8,Pazar
1,10537.000,8.000,TOZ SEKER,18105,2017-01-06,5.000,2.650,13.250,12.270,8.000,Antalya Subesi,İlhan ÖRENLİ,Antalya,Akdeniz,36.884,30.706,131464,İsmet ŞINGIR,37,DOĞADAN,İÇECEK,ÇAY KAHVE,SEKER TATLANDIRICI,2017-01-07 11:04:34,2017-01-07 11:05:37,E,2017,Ocak,7,Cumartesi
2,11335.000,5979.000,FALIM SAKIZ 5LI NANE,18350,2017-01-03,1.000,0.400,0.400,0.370,40.000,İstanbul Subesi,İsmet SARTIK,İstanbul,Marmara,41.005,28.977,656969,Yağız KUBAL,300,FALIM,GIDA,SAKIZ SEKERLEME,SAKIZ,2017-01-04 14:00:03,2017-01-04 14:01:01,E,2017,Ocak,4,Çarşamba
3,11336.000,5979.000,FALIM SAKIZ 5LI NANE,18350,2017-01-03,1.000,0.400,0.400,0.370,40.000,İstanbul Subesi,İsmet SARTIK,İstanbul,Marmara,41.005,28.977,656969,Yağız KUBAL,300,FALIM,GIDA,SAKIZ SEKERLEME,SAKIZ,2017-01-04 14:00:03,2017-01-04 14:01:01,E,2017,Ocak,4,Çarşamba
4,10115.000,12808.000,FALIM SAKIZ 5LI CILEK,18005,2017-01-05,1.000,0.400,0.400,0.370,41.000,İzmir Subesi,Gülten IŞLAK,İzmir,Ege,38.419,27.129,447336,Habibe AYSAN,300,FALIM,GIDA,SAKIZ SEKERLEME,SAKIZ,2017-01-06 14:00:30,2017-01-06 14:01:03,K,2017,Ocak,6,Cuma


In [17]:
df2.shape

(562201, 30)

In [18]:
df2.isnull().sum()

ID                      0
ÜrünKodu                0
ÜrünAdı                 0
FişNo                   0
Tarih                   0
Miktar                  0
Fiyat                   0
ToplamNetSatış          0
NetSatış                0
ŞubeNo                  0
Şube                    0
SatışElemanı            0
Şehir                   0
Bölge                   0
Enlem                   0
Boylam                  0
MüşteriKodu             0
MüşteriAdı              0
MarkaKodu               0
Marka                   0
KategoriAdı1            0
KategoriAdı2            0
KategoriAdı3       165941
BaşlangıçTarihi         0
BitişTarihi             0
Cinsiyet                0
Yil                     0
Ay                      0
Gun                     0
HaftaninGunu            0
dtype: int64

In [ ]:
kate